# Using Stochastic Gradient Descent On Linear Regression To Predict The Best Time To Go To The Gym

## 1. Introduction

We will use the "Crowdedness at the Campus Gym" dataset and the stochastic gradient descent linear regression (SGDRegressor) algorithm from the scikit-learn library to build and train a model for predicting the best time to go to the gym and avoid crowds.

The dataset contains 11 columns with the following information.

* `number_people`: this is the number of people at the gym at each observation. This will be our target variable or label.